In [14]:
# max: 3x + y

# 0 ≤ x ≤ 1
# 0 ≤ y ≤ 2
# x + y ≤ 2

# Importe as bibliotecas necessárias
from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit

# Declare o solucionador.
    # Create the linear solver with the GLOP backend.
solver = pywraplp.Solver.CreateSolver('GLOP')


# Crie as variáveis
    # Create the variables x and y.
x = solver.NumVar(0, 1, 'x')
y = solver.NumVar(0, 2, 'y')

print('Number of variables =', solver.NumVariables())


# Defina as restrições,
    # Create a linear constraint, 0 <= x + y <= 2.
ct = solver.Constraint(0, 2, 'ct')
ct.SetCoefficient(x, 1)
ct.SetCoefficient(y, 1)

print('Number of constraints =', solver.NumConstraints())

# Defina a função de objetivo
    # Create the objective function, 3 * x + y.
objective = solver.Objective()
objective.SetCoefficient(x, 3)
objective.SetCoefficient(y, 1)
objective.SetMaximization()

# Invoque o solucionador e exiba os resultados.
solver.Solve()
print('Solution:')
print('Objective value =', objective.Value())
print('x =', x.solution_value())
print('y =', y.solution_value())

Number of variables = 2
Number of constraints = 1
Solution:
Objective value = 4.0
x = 1.0
y = 1.0


In [42]:
from ortools.linear_solver import pywraplp

# Define the problem data
num_nodes = 10
prize_val = [20, 25, 18, 22, 19, 23, 20, 21, 24, 20]
penalty_val = [5, 6, 4, 5, 5, 6, 5, 5, 6, 5]
quota = 70

# Create the solver
solver = pywraplp.Solver.CreateSolver('SCIP')

# Create the decision variables
x = {}
for i in range(num_nodes):
    for j in range(num_nodes):
        if i != j:
            x[i, j] = solver.BoolVar('x[%i,%i]' % (i, j))

# Create the objective function
objective = solver.Objective()
for i in range(num_nodes):
    for j in range(num_nodes):
        if i != j:
            objective.SetCoefficient(x[i, j], prize_val[i])
objective.SetMinimization()

# Create the constraints
for i in range(num_nodes):
    expr = solver.Sum([x[i, j] for j in range(num_nodes) if j != i])
    solver.Add(expr == 1)
    
for j in range(num_nodes):
    expr = solver.Sum([x[i, j] for i in range(num_nodes) if i != j])
    solver.Add(expr == 1)

for i in range(num_nodes):
    for j in range(num_nodes):
        if i != j:
            solver.Add(x[i, j] + x[j, i] <= 1)

# Add penalty constraints
penalty = {}
for i in range(num_nodes):
    for j in range(num_nodes):
        if i != j:
            penalty[i, j] = penalty_val[i]
            penalty[j, i] = penalty_val[j]
            
for i in range(num_nodes):
    for j in range(num_nodes):
        if i != j:
            solver.Add(quota - solver.Sum([prize_val[k]*x[k, l].solution_value() for k in range(num_nodes) for l in range(num_nodes) if (k,l) in x.keys() and (k == i or l == i) and (k == j or l == j)]) >= penalty[i, j])

# Solve the problem
status = solver.Solve()

# Print the solution
if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                if x[i, j].solution_value() > 0.5:
                    print('Edge (%i, %i) is part of the optimal tour.' % (i, j))
else:
    print('The problem does not have an optimal solution')


Objective value = 212.0
Edge (0, 1) is part of the optimal tour.
Edge (1, 2) is part of the optimal tour.
Edge (2, 0) is part of the optimal tour.
Edge (3, 4) is part of the optimal tour.
Edge (4, 5) is part of the optimal tour.
Edge (5, 3) is part of the optimal tour.
Edge (6, 7) is part of the optimal tour.
Edge (7, 8) is part of the optimal tour.
Edge (8, 9) is part of the optimal tour.
Edge (9, 6) is part of the optimal tour.
